In [ ]:
pip install geopandas
pip install folium
pip install geoplot
pip install pysal
pip install pandas
pip install shapely

import geoplot as gplt
from geopandas import GeoDataFrame
import geopandas as gpd
import geoplot.crs as gcrs
import imageio
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import mapclassify as mc
import numpy as np

import boto3
import pandas as pd
import io

s3 = boto3.resource('s3')
bucket = s3.Bucket('Bucket_name')
prefix_objs = bucket.objects.filter(Prefix="run-AmazonS3_node")

prefix_df = []

for obj in prefix_objs:
    key = obj.key
    body = obj.get()['Body'].read()
    temp = pd.read_parquet(io.BytesIO(body))        
    prefix_df.append(temp)
df = pd.concat(prefix_df)
df.head()

#combine lat and lon
df['geo'] = df[['latitude', 'longtitude']].apply(lambda x: ' , '.join(x), axis=1)
df.head()

# Getting a 2 point geometry
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df.longtitude, df.latitude)]
#df21 = df15.drop(['lon', 'lat'], axis=1)
df = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)
gdf = df[['geometry']]
gdf.head()

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# Create a subset of the GeoDataFrame
#france = world[world.name == "France"]
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15);

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# Create a subset of the GeoDataFrame
#france = world[world.name == "France"]
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15)

U#Using plotly for visualization
import plotly.express as px

color_scale = [(0, 'orange'), (1,'red')]
df.latitude = df.latitude.astype(float)
df.longtitude = df.longtitude.astype(float)
fig = px.scatter_mapbox(df, 
                        lat="latitude", 
                        lon="longtitude", 
                        hover_name="restaurant_name", 
                        #hover_data=["Address", "Listed"],
                        color="place_id",
                        color_continuous_scale=color_scale,
                        #size="Listed",
                        zoom=8, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
#fig.update_layout(mapbox_style="google-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#Using filium for visualization
import folium 
map = folium.Map(location=[48.864716, 2.349014], zoom_start=13)
map

points = [[lat, lon] for lat, lon in zip(df['latitude'], df['longtitude'])]
print(points)

for point in points:
    folium.Marker(point, popup=names).add_to(map)
map